In [1]:
import struct
print(struct.calcsize('P')*8)

32


In [1]:
# ver2 수정사항 : SQL 처리속도가 느려 메모리 누적현상 발생으로 매도,매수잔량을 featurecode : 3을 사용하여 한번에 전송함
# ver3 수정사항 : SQL대신 로컬 csv로 저장 시도 / sql_que 내용중 앞쪽 1개 내용 로그창에 표시 -> 안됨
# ver4 수정사항 : 테마 정보 처리

import win32com.client
import time
import datetime
import sys
import pythoncom
import pymysql
# import sql
import threading
import os

from multiprocessing import Process, Queue
from PyQt5.QtWidgets import *
from PyQt5 import uic
from PyQt5.QtGui import QStandardItemModel
from PyQt5.QtGui import QStandardItem
from PyQt5.QtCore import *

import pandas as pd

import stop_code

form_class = uic.loadUiType("main.ui")[0]
sql_que = []
sql_feature_code_dict = {3:'매수잔량', 4:'매도잔량', 6:'체결강도', 8:'누적거래량', 9:'코스피지수', 10:'테마', 20:'3일평균,총주식수'}
sql_income_num = 0
sql_complete_num = 0


def InitPlusCheck():
    # 프로세스가 관리자 권한으로 실행 여부
    if ctypes.windll.shell32.IsUserAnAdmin():
        print('정상: 관리자권한으로 실행된 프로세스입니다.')
    else:
        print('오류: 일반권한으로 실행됨. 관리자 권한으로 실행해 주세요')
        return False
 
    # 연결 여부 체크
    if (g_objCpStatus.IsConnect == 0):
        print("PLUS가 정상적으로 연결되지 않음. ")
        return False
 
    return True

# sql로 저장
def sql_start(sql_que) :
    sql_addr = 'localhost'
    sql_user = 'root'
    sql_pw = '1234'
    # sql_addr = '14.35.220.97'
    # sql_user = 'seominseok'
    # sql_pw = 'pintech801'

    print('sql Process start...')
    # sql = pymysql.connect(host = 'localhost',port=3306, user='root', passwd='1234', db='dbstock', charset='utf8')
    sql = pymysql.connect(host = sql_addr, port=3306, user=sql_user, passwd=sql_pw, db='dbstock', charset='utf8')
    cursor = sql.cursor()

    global sql_complete_num
    while True :
        if len(sql_que) == 0 :
            time.sleep(1)
            # print('Empty Queue... Wait 1 second')
            continue
        
        popped_item = sql_que.pop(0)
        if len(popped_item)==4 :
            code_, feature_code_, time_, value_ = popped_item
            # print('4개 : ', code_,'/', feature_code_, '/',time_, '/',value_)
        elif len(popped_item)==5 :
            # feature_code_ = int(popped_item[1])
            # print('feature_code : ', feature_code_)
            # if (feature_code_ == 3) | (feature_code_ == 4) :
                # print('feature code : 3 or 4')
            code_, feature_code_, time_, buy_remain_value_, sell_remain_value_ = popped_item
        elif len(popped_item)==6 :
            # if feature_code_ == 20 :
            code_, feature_code_, time_, item_code_, days_3_avr_, total_stocks_ = popped_item
            # print('5개 : ', code_,'/', feature_code_, '/',time_,'/', buy_remain_value_, '/',sell_remain_value_)

        # print(code_, feature_code_, time_, value_)

        #3일 평균 및 총 주식 발행수를 별도로 처리해 주기 위해 if문 추가
        if feature_code_ != 20 :
            # time 을 읽어와서 db입력 형식으로 변환해 줌
            time_ = str(time_)

            time_time_ = int(time_[:-2])
            time_min_ = int(time_[-2:])

            time_date_ = datetime.datetime.today().strftime('%Y-%m-%d')
            if time_time_ == 24 :
                continue

            time_final_ = time_date_+' '+str(time_time_).zfill(2)+':'+str(time_min_).zfill(2)

        
            # 종목명의 테이블이 있는지 확인
            sql_comm_ = "show tables like '{}'".format(code_)
            cursor.execute(sql_comm_)
            return_ = cursor.fetchall()
            # print('sql que : check table : ', return_)
            # 없으면 테이블 생성
            if len(return_)==0 :
                try :
                    sql_comm_ = '''
                    CREATE TABLE {} (
                    `체결시간` DATETIME NOT NULL,
                    `매수잔량` DOUBLE NULL,
                    `매도잔량` DOUBLE NULL,
                    `체결강도` DOUBLE NULL,
                    `누적거래량` DOUBLE NULL,
                    PRIMARY KEY (`체결시간`),
                    UNIQUE INDEX `time_UNIQUE` (`체결시간` ASC));
                    '''.format(code_)
                    cursor.execute(sql_comm_)
                    return_ = cursor.fetchall()
                    sql.commit()
                    print("CREATE TABLE {} ".format(code_))
                    print('SQL thread : DB 생성 완료')
                except Exception as e :
                    print('SQL thread : DB 생성 오류, ',e)
                    sql = pymysql.connect(host = sql_addr, port=3306, user=sql_user, passwd='1234', db='dbstock', charset='utf8')
                    cursor = sql.cursor()
                    continue

            # 컬럼이 있는지 확인 함        
            sql_comm_ = "SHOW COLUMNS FROM {} LIKE '{}';".format(code_,sql_feature_code_dict[feature_code_])
            cursor.execute(sql_comm_)
            return_ = cursor.fetchall()
            # print('sql que : check column : ', return_)
            if len(return_)==0 :
                try :
                    # 컬럼이 없을때 생성
                    sql_comm_ = "ALTER TABLE {} ADD {} DOUBLE;".format(code_,sql_feature_code_dict[feature_code_])
                    cursor.execute(sql_comm_)
                    sql.commit()
                    return_ = cursor.fetchall()
                except Exception as e :
                    print('SQL error : column 생성 실패')
                    print(e)
                    sql = pymysql.connect(host = sql_addr, port=3306, user=sql_user, passwd='1234', db='dbstock', charset='utf8')
                    cursor = sql.cursor()
                    continue
            # 있으면 데이터 입력
            try : 
                # if feature_code_ != 3 :
                #     sql_comm_ = "select {} is not NULL from {} where 체결시간='{}';".format(sql_feature_code_dict[feature_code_],code_,str(time_final_))
                #     cursor.execute(sql_comm_)
                #     return_ = cursor.fetchall()
                # if return_[0][0]==1 :
                #     print("select {} is not NULL from {} where 체결시간='{}';".format(sql_feature_code_dict[feature_code_],code_,str(time_final_)))
                #     continue

                if feature_code_==3 :
                    # print('데이터 2개 신규구나...')
                    sql_comm_ = "INSERT INTO {} (체결시간, 매수잔량, 매도잔량) VALUES ('{}','{}','{}');".format(str(code_),str(time_final_),buy_remain_value_,sell_remain_value_)
                else :
                    # print('데이터 1개 신규')
                    sql_comm_ = "INSERT INTO {} (체결시간, {}) VALUES ('{}','{}');".format(str(code_),str(sql_feature_code_dict[feature_code_]),str(time_final_),str(value_))
                cursor.execute(sql_comm_)
                # print('DB  정상입력')
                return_ = cursor.fetchall()
                # print('1 : ',return_)
                sql.commit()
                sql_complete_num+=1
            except Exception as e:
                if feature_code_==3 :
                    # print('데이터 2개 수정이구나...')
                    sql_comm_ = "UPDATE {} SET {}='{}',{}='{}' WHERE 체결시간='{}';".format(str(code_),'매수잔량',str(buy_remain_value_),'매도잔량',str(sell_remain_value_),str(time_final_))
                else :
                    # print('데이터 1개 수정.')
                    sql_comm_ = "UPDATE {} SET {}='{}' WHERE 체결시간='{}';".format(str(code_),str(sql_feature_code_dict[feature_code_]),str(value_),str(time_final_))
                cursor.execute(sql_comm_)
                # print('DB  중복감지',e)
                return_ = cursor.fetchall()
                # print('2 : ',return_)
                sql.commit()
                sql_complete_num+=1
                continue
        
        # 3일 평균 및 총 주식 발행수 처리
        else : 
            # 종목명의 테이블이 있는지 확인
            sql_comm_ = "show tables like '{}'".format(code_)
            cursor.execute(sql_comm_)
            return_ = cursor.fetchall()
            # print('sql que : check table : ', return_)
            # 없으면 테이블 생성
            if len(return_)==0 :
                try :
                    sql_comm_ = '''
                    CREATE TABLE {} (
                    `체결시간` DATETIME NOT NULL,
                    `종목코드` CHAR(7) NULL,
                    `종목명` varchar(20) NULL,
                    `3일평균가` DOUBLE NULL,
                    `발행주식수` DOUBLE NULL);
                    '''.format(code_)
                    cursor.execute(sql_comm_)
                    return_ = cursor.fetchall()
                    sql.commit()
                    print("CREATE TABLE {} ".format(code_))
                    print('SQL thread : DB 생성 완료')
                except Exception as e :
                    print('SQL thread : DB 생성 오류, ',e)
                    sql = pymysql.connect(host = sql_addr, port=3306, user=sql_user, passwd='1234', db='dbstock', charset='utf8')
                    cursor = sql.cursor()
                    continue

            
            # 데이터 입력
            try : 
                # print('평균가, 총주식수 데이터 신규...')
                sql_comm_ = "INSERT INTO {} (체결시간, 종목코드, 3일평균가, 발행주식수) VALUES ('{}','{}','{}','{}');".format(code_, time_, item_code_, days_3_avr_, total_stocks_)
                cursor.execute(sql_comm_)
                return_ = cursor.fetchall()
                sql.commit()
                sql_complete_num+=1
            except Exception as e:
                print('sql thread : 평균가, 총주식수 DB 입력 error!! : ', e)
                continue


# class Thread1(QThread):
#     def __init__(self, parent) :
#         print('thread : initiate')
#         super().__init__(parent)
#         self.parent = parent
    
#     def run(self) :
#         # pythoncom.CoInitialize()


class Thread1(QThread):
    def __init__(self, parent) :
        print('thread : initiate')
        super().__init__(parent)
        self.parent = parent

    def run(self) :
        pythoncom.CoInitialize()
        info = win32com.client.Dispatch('CpSysDib.CpSvr7066')
        strengthen = win32com.client.Dispatch('Dscbo1.CpSvr8083')
        stockmst = win32com.client.Dispatch('Dscbo1.StockMst')
        secter = win32com.client.Dispatch('Dscbo1.CpSvr8562')
        print('thread : run')

        cnt=0
        

        while True :
            # start_time_ = datetime.datetime.now()
            # print('Thread1 : entering loop')
            try :
                # print('Thread1 : in the loop : cnt : ', cnt)
                text_ = self.parent.tableWidget.item(cnt,0)
                text_ = text_.text()
                # print(text_)
                # self.parent.log_Que.append(text_)
                info.SetInputValue(0,text_)
                self.parent.log_Que.append('Set Code...{}'.format(text_))
                info.SetInputValue(1,1)
                info.SetInputValue(2,ord('0'))

                # status = info.GetDibStatus()
                # self.parent.log_Que.append('통신상태 : , {}, {}'.format(status, info.GetDibMsg1()))
                info.BlockRequest()
                time.sleep(self.parent.txt_sleep.value())

                strengthen.SetInputValue(0,text_)
                strengthen.SetInputValue(1,'1')
                strengthen.BlockRequest()
                time.sleep(self.parent.txt_sleep.value())

                stockmst.SetInputValue(0,text_)
                stockmst.BlockRequest()
                time.sleep(self.parent.txt_sleep.value())

                value = info.GetHeaderValue(1)
                # self.parent.display_log('get header value 2')

                status = info.GetDibStatus()
                # self.parent.log_Que.append('통신상태 : , {}, {}'.format(status, info.GetDibMsg1()))

                remain_time_list = []
                str_time_list = []
                mst_time_list = []
                
                flag = True
                global sql_income_num
                # 매도매수잔량
                final_remain_list = []
                for i in range(info.GetHeaderValue(1)) :
                    try :
                        time_ = info.GetDataValue(0,i)
                        buy_remain_value = info.GetDataValue(5,i)
                        sell_remain_value = info.GetDataValue(6,i)

                        remain_time_list.append(int(time_))

                        # sql_que.append([text_, 3, time_, buy_remain_value])
                        # sql_que.append([text_, 4, time_, sell_remain_value])

                        sql_que.append([text_, 3, time_, buy_remain_value, sell_remain_value])
                        sql_income_num+=1

                        if flag :
                            self.parent.main_display_Que.append([cnt,2,time_])
                            self.parent.main_display_Que.append([cnt, 3, buy_remain_value])
                            self.parent.main_display_Que.append([cnt, 4, sell_remain_value])
                            flag=False
                    except Exception as e:
                        print('Thread : info GetDataValue : ',text_, time_, buy_remain_value, sell_remain_value,e)
                        continue
                sql_que.append(final_remain_list)
                # 채결강도
                flag = True
                
                for i in range(strengthen.GetHeaderValue(0)) :
                    try :
                        str_time_ = strengthen.GetDataValue(0,i)
                        str_value_ = strengthen.GetDataValue(1,i)

                        str_time_list.append(int(str_time_))

                        # sql_que.append([text_, 5, str_time_])
                        sql_que.append([text_, 6, str_time_, str_value_])
                        sql_income_num+=1

                        if flag :
                            self.parent.main_display_Que.append([cnt, 5, str_time_])
                            self.parent.main_display_Que.append([cnt, 6, str_value_])
                            flag = False
                    except Exception as e:
                        print('Thread : strengthen GetDataValue : ',text_, str_time_, str_value_,e)
                        continue
                
                #데이터 추가 후 refresh가 안되서 추가 함
                self.parent.tableWidget.viewport().update()

                cnt+=1

                if cnt == self.parent.tableWidget.rowCount() :
                    self.parent.read_kospi_index()
                    cnt=0

            except Exception as e :
                print('Thread1 error : ',e)
                self.parent.log_Que.append('Thread1 error : {}'.format(e))
                cnt=0
                continue

            

class timer_Thread(QThread):
    def __init__(self, parent) :
        super().__init__(parent)
        self.parent = parent

    def run(self) :
        while True :
            # print('timer thread start')
            self.parent.sql_mem_indi.display(len(sql_que))
            self.parent.sql_complete_indi.display(sql_complete_num)
            self.parent.sql_income_indi.display(sql_income_num)
            time.sleep(2)




class log_Thread(QThread):
    def __init__(self, parent) :
        super().__init__(parent)
        self.parent = parent
        self.list_ = QStandardItemModel()
        print('log_thread : Initiate')

    def run(self) :
        print('log_thread : Run')
        timer = time.localtime()
        while True :
            if len(self.parent.log_Que) > 0 :
                log_data = self.parent.log_Que.pop(0)

                self.list_.appendRow(QStandardItem(log_data))
                self.parent.log_list.setModel(self.list_)
            else :
                time.sleep(1)

class main_display_Thread(QThread):
    def __init__(self, parent) :
        super().__init__(parent)
        self.parent = parent
        print('main_display_thread : Initiate')

    def run(self) :
        print('main_display_thread : Run')
        while True :
            if len(self.parent.main_display_Que) > 0 :
                r,c,txt = self.parent.main_display_Que.pop(0)
                self.parent.tableWidget.setItem(r, c, QTableWidgetItem(str(txt)))
                
            else :
                time.sleep(0.5)

class main_window(QMainWindow, form_class) :
    

    def __init__(self):
        super().__init__()
        print('main_window : initiate window')
        self.main_display_Que = []
        self.log_Que = []
        self.transfer_Que = []

        self.InitUI()
        self.get_real_data.clicked.connect(self.run_real_data)
        self.load_stop_codes.clicked.connect(self.read_stop_code)
        self.load_kospi_index.clicked.connect(self.read_kospi_index)
        self.load_thema_data.clicked.connect(self.read_sector)
        self.load_avr_total.clicked.connect(self.read_avr_total)
        self.thread1 = Thread1(self)
        self.timer_thread = timer_Thread(self)
        self.log_thread = log_Thread(self)
        self.main_display_thread = main_display_Thread(self)
        self.log_thread.start()
        self.main_display_thread.start()
        self.timer_thread.start()

    def read_kospi_index(self) :
        kospi_index = win32com.client.Dispatch('Dscbo1.StockIndexIR')
        
        kospi_index.SetInputValue(0,'U001')
        kospi_index.BlockRequest()
        kospi_num_ = kospi_index.GetHeaderValue(1)
        
        self.kospi_index_table.setRowCount(kospi_num_)
        self.kospi_index_table.setColumnCount(2)

        global sql_income_num
        for i in range(kospi_num_) :
            time_ = kospi_index.GetDataValue(0,i)
            kospi_ = kospi_index.GetDataValue(1,i)

            # print('{}\t{}\t{}'.format(i,time_, kospi_))

            self.kospi_index_table.setItem(i, 0, QTableWidgetItem(str(time_)))
            self.kospi_index_table.setItem(i, 1, QTableWidgetItem(str(kospi_)))

            sql_que.append(['U001', 9, time_, kospi_])
            sql_income_num+=1

    # 3일 평균가, 총 주식 발행수 를 데이터베이스에 쓰기 위해 z001 테이블 별도로 생성하여 기록 함
    # 임의로 체결시간은 표기일의 15:30으로 사용 함
    def read_avr_total(self) :

        avr_days = 3

        avr_total_ = win32com.client.Dispatch('CpSysDib.StockChart')

        df = pd.DataFrame(None, columns=['time','code','avr','total'])
        
        for i in range(self.tableWidget.rowCount()) :
            text_ = self.tableWidget.item(i,0)
            text_ = text_.text()

            
            avr_total_.SetInputValue(0,text_)
            avr_total_.SetInputValue(1,ord('2'))
            avr_total_.SetInputValue(4,avr_days)
            avr_total_.SetInputValue(5,[0,5,12])
            avr_total_.SetInputValue(6,ord('D'))

            avr_total_.BlockRequest()

            code_ = avr_total_.GetHeaderValue(0)
            num_ = int(avr_total_.GetHeaderValue(3))
            field_name_ = avr_total_.GetHeaderValue(2)
            field_num_ = int(avr_total_.GetHeaderValue(1))

            price_sum_ = 0

            for j in range(avr_days) :
                price_sum_ += int(avr_total_.GetDataValue(1,j))
            avr_price_ = price_sum_/avr_days

            time_ = str(avr_total_.GetDataValue(0,0))
            time_year_ = time_[:4]
            time_mon_ = time_[4:6]
            time_day_ = time_[6:]
            time_date_ = time_year_ + '-' + time_mon_ + '-' + time_day_ 
            time_final_ = time_date_+' 15:30'

            stock_total_ = avr_total_.GetDataValue(2,0)

            sql_que.append(['Z001', 20, time_final_, code_, avr_price_, stock_total_])
            # print(['Z001', 20, time_final_, code_, avr_price_, stock_total_])

            new_item_ = {'time':time_final_,'code':code_,'avr':avr_price_,'total':stock_total_}
            df = df.append(new_item_, ignore_index=True)

            self.display_avr_total_data(df)
            

            time.sleep(self.txt_sleep.value())

    def display_avr_total_data(self, df) :

        self.avr_total_table.setRowCount(len(df))
        self.avr_total_table.setColumnCount(len(df.iloc[0]))

        # QTableWidget 에서 overflow 발생으로 str형식으로 변경 후 데이터 표시
        df['avr'] = df['avr'].astype(str)
        df['total'] = df['total'].astype(str)

        for i in range(len(df)) :
            for j in range(len(df.iloc[i])) :
                self.avr_total_table.setItem(i, j, QTableWidgetItem(df.iloc[i,j]))

                

    def read_kospi(self, stop_name) :
        self.code_list = win32com.client.Dispatch("CpUtil.CpCodeMgr")
        self.codeList = self.code_list.GetStockListByMarket(1)
        self.kospiList = []
        # self.kospiList.append('U001')

        for code in self.codeList:
            secondCode = self.code_list.GetStockSectionKind(code)
            name = self.code_list.CodeToName(code)
            # print(i, code, secondCode, name)
            if ((secondCode == 1)&(name not in stop_name)):
                self.kospiList.append(code)
        
        self.display_code()


    def display_code(self) :
        
        for cnt,code in enumerate(self.kospiList) :

            name = self.code_list.CodeToName(code)

            self.tableWidget.setRowCount(len(self.kospiList))

            self.main_display_Que.append([cnt,0,str(code)])
            self.main_display_Que.append([cnt,1,name])




    def display_thema_data(self, df) :

        self.thema_table.setRowCount(len(df))
        self.thema_table.setColumnCount(len(df.iloc[0]))

        for i in range(len(df)) :
            for j in range(len(df.iloc[i])) :
                self.thema_table.setItem(i, j, QTableWidgetItem(df.iloc[i,j]))

    

    def read_stop_code (self) :
        df_stop_code = pd.read_csv('./stop_code.csv')
        stop_name = []
        # print('df_stop_code : ',df_stop_code.iloc[i,1])

        for i in range(len(df_stop_code)):
            # print('i : ',i)
            # self.display_log('df_stop_code[i][1] : ',df_stop_code.iloc[i,1])
            stop_name.append(df_stop_code.iloc[i,1])
            self.stop_code_table.setItem(i, 0, QTableWidgetItem(df_stop_code.iloc[i,1]))
        
        self.read_kospi(stop_name)

    def read_sector(self) :
        secter_code = win32com.client.Dispatch('Dscbo1.CpSvr8561')     
        sector_item = win32com.client.Dispatch('Dscbo1.CpSvr8561T')

        df_thema = pd.DataFrame(None, columns=['thema code','thema order','thema name','item code','item name'])
        secter_code.BlockRequest() 

        num_ = secter_code.GetHeaderValue(0)
        print('num : ',num_)
        print('code \t order \t name')     

        for i in range(num_) :

            code_ = secter_code.GetDataValue(0,i)
            order_ = secter_code.GetDataValue(1,i)
            name_ = secter_code.GetDataValue(2,i)

            sector_item.SetInputValue(0,code_)
            sector_item.BlockRequest()

            item_num_ = sector_item.GetHeaderValue(1)

            for j in range(item_num_) :
                
                item_code_ = sector_item.GetDataValue(0,j)
                item_name_ = sector_item.GetDataValue(1,j)
                
                new_item = {'thema code': str(code_),'thema order':str(order_),'thema name':name_,'item code':item_code_,'item name':item_name_}

                df_thema = df_thema.append(new_item, ignore_index=True)
            
            time.sleep(self.txt_sleep.value())
                
        # print(df_thema)
        self.display_thema_data(df_thema)   
        df_thema.to_csv('thema_data.csv', encoding='utf8') 
        

    def InitUI(self) :
        self.setupUi(self)
        print('main_window : initiate UI')

        self.read_stop_code()
        # self.read_sector()

        exitAction = QAction('Exit', self)
        exitAction.setShortcut('Ctrl+Q')
        exitAction.setStatusTip('Exit application')
        exitAction.triggered.connect(self.app_exit)

        # open_stop_code = QAction('Open stop code', self)
        # open_stop_code.setShortcut('Ctrl+A')
        # open_stop_code.setStatusTip('Open stop codes')
        # open_stop_code.triggered.connect(self.open_stop_code)

        menubar = self.menuBar()
        menubar.setNativeMenuBar(False)
        fileMenu = menubar.addMenu('&File')
        fileMenu.addAction(exitAction)
        # fileMenu.addAction(open_stop_code)
        
        # self.txt_sleep.setText(str(0.5))
        

    # def open_stop_code(self) :
    #     stop_code_app = QApplication(sys.argv)
    #     stop_window = stop_code.stop_code()
    #     print('stop_code_window : Initiate')
    #     stop_window.show()
    #     print('show window')
    #     stop_code_app.exec_()


    def app_exit(self):
        print('Quit the App')
        app.quit()
        sys.exit()

    def run_real_data(self) :
        self.thread1.start()
        
        
        
        


if __name__ == '__main__' :
    # sql_que = Queue()
    app = QApplication(sys.argv)
    t_sql = threading.Thread(target=sql_start, args=(sql_que,), daemon=True)
    t_sql.start()
    window = main_window()
    print('main_window : Initiate')
    window.show()
    print('show window')
    app.exec_()






sql Process start...
main_window : initiate window
main_window : initiate UI
thread : initiate
log_thread : Initiate
main_display_thread : Initiate
main_window : Initiate
show windowmain_display_thread : Run

log_thread : Run
thread : run
0	2400	2988.77
1	2359	2988.77
2	1530	2988.77
3	1529	2988.8
4	1528	2988.8
5	1527	2988.8
6	1526	2988.8
7	1525	2988.8
8	1524	2988.8
9	1523	2988.8
10	1522	2988.8
11	1521	2988.8
12	1520	2988.8
13	1519	2989.29
14	1518	2989.02
15	1517	2988.43
16	1516	2988.21
17	1515	2986.76
18	1514	2987.81
19	1513	2987.3
20	1512	2987.94
21	1511	2988.13
22	1510	2988.01
23	1509	2988.12
24	1508	2987.54
25	1507	2988.32
26	1506	2988.96
27	1505	2988.15
28	1504	2987.72
29	1503	2987.59
30	1502	2986.49
31	1501	2985.17
32	1500	2987.76
33	1459	2985.11
34	1458	2986.34
35	1457	2987.26
36	1456	2986.74
37	1455	2987.21
38	1454	2986.95
39	1453	2986.58
40	1452	2986.7
41	1451	2988.23
42	1450	2988.05
43	1449	2988.1
44	1448	2987.4
45	1447	2987.72
46	1446	2988.18
47	1445	2987.73
48	1444	2988.44
4

In [1]:
%time

import win32com.client
info = win32com.client.Dispatch('CpSysDib.CpSvr7066')


info.SetInputValue(0,'A005930')
info.SetInputValue(1,1)
print('set input value 1')
info.SetInputValue(2,ord('0'))
print('set input value 2')

info.BlockRequest()
status = info.GetDibStatus()
print('통신상태 : , {}, {}'.format(status, info.GetDibMsg1()))
                
print('block request')

value = info.GetHeaderValue(1)
print('get header value 2')

status = info.GetDibStatus()
print('get dib status')

print('통신상태 : , {}, {}'.format(status, info.GetDibMsg1()))

print('value : ',value)


for i in range(20):
    try : 
        time_ = info.GetDataValue(0,i)
        value = info.GetDataValue(5,i)
        print('time : {}\t value : {}'.format(time_,value))
    except :
        break

Wall time: 0 ns
set input value 1
set input value 2
통신상태 : , 0, B035 조회가 계속 됩니다.(stock.svr.7066cbp)
block request
get header value 2
get dib status
통신상태 : , 0, B035 조회가 계속 됩니다.(stock.svr.7066cbp)
value :  15
time : 948	 value : 1358858
time : 947	 value : 1294299
time : 946	 value : 1296809
time : 945	 value : 1295831
time : 944	 value : 1366248
time : 943	 value : 1598578
time : 942	 value : 1590862
time : 941	 value : 1375110
time : 940	 value : 1302538
time : 939	 value : 1294831
time : 938	 value : 1423773
time : 937	 value : 1181674
time : 936	 value : 1174774
time : 935	 value : 1061280
time : 934	 value : 1084825


In [1]:
import win32com.client

test = win32com.client.Dispatch('Dscbo1.StockAdR')

# test.SetInputValue(0,'1')
status = test.GetDibStatus()
value = test.GetHeaderValue(5)

print('status : ',status)
print('value : ',value)

status :  0
value :  0.0


In [ ]:
code_list = win32com.client.Dispatch("CpUtil.CpCodeMgr")
codeList = code_list.GetStockListByMarket(1)
print(codeList)

In [1]:
import win32com.client
strengthen = win32com.client.Dispatch('Dscbo1.CpSvr8083')

strengthen.SetInputValue(0,'A005930')
strengthen.SetInputValue(1,'1')

strengthen.BlockRequest()

cnt = strengthen.GetHeaderValue(0)
print('cnt : ', cnt)
value = strengthen.GetDataValue(8,0)

print('value : ',value)

cnt :  381
value :  1151702


In [4]:
import win32com.client
stockmst = win32com.client.Dispatch('Dscbo1.StockMst')

stockmst.SetInputValue(0,'A005930')

stockmst.BlockRequest()

code_ = stockmst.GetHeaderValue(0)
name_ = stockmst.GetHeaderValue(1)
time_ = stockmst.GetHeaderValue(4)
accum_purc_ = stockmst.GetHeaderValue(19)
print('code : ', code_)
print('name : ', name_)
print('time : ', time_)
print('accum_purc : ', accum_purc_)
value = stockmst.GetDataValue(2,1)

print('value : ',value)



code :  A005930
name :  삼성전자
time :  1555
accum_purc :  161853823
value :  6775


In [34]:
import win32com.client
gigwan = win32com.client.Dispatch('CpSysDib.CpSvrNew7222')

gigwan.SetInputValue(0,ord('B'))
gigwan.SetInputValue(1,0)
gigwan.SetInputValue(2,'1')
gigwan.SetInputValue(4,'A005930')

gigwan.BlockRequest()

num_ = int(gigwan.GetHeaderValue(0))

print('time \t gain \t fore \t gigwan')

for i in range(num_) :
    time_ = gigwan.GetDataValue(0,i)
    gain_ = gigwan.GetDataValue(1,i)
    fore_ = gigwan.GetDataValue(2,i)
    gigwan_ = gigwan.GetDataValue(3,i)

    print('{}\t{}\t{}\t{}'.format(time_,gain_,fore_,gigwan_))

    # print('time : ', time_)
    # print('gain : ', gain_)
    # print('fore : ', fore_)
    # print('gigwan : ', gigwan_)
    
# value = stockmst.GetDataValue(2,1)

# print('value : ',value)


com_error: (-2147352567, '예외가 발생했습니다.', (0, None, 'Module:CpSysDib\nObj:CpSvrNew7222\nFun:SetInputValue\nType:4\nDescription:형식이 일치하지 않습니다.', None, 0, -2147024809), None)

In [23]:
import win32com.client
thema = win32com.client.Dispatch('Dscbo1.CpSvr8562')
# thema = win32com.client.Dispatch('Dscbo1.CpSvr8561')

# gigwan.SetInputValue(0,'B')
# gigwan.SetInputValue(1,0)
# gigwan.SetInputValue(2,'1')
thema.SetInputValue(0,'A000060')

thema.BlockRequest()

num_ = thema.GetHeaderValue(0)

print('num : ',num_)
thema_code_ = thema.GetDataValue(0,0)
order_ = thema.GetDataValue(1,0)
name_ = thema.GetDataValue(2,0)

print('thema_code : ',code_)
print('code : ','A000060')
print('order : ',order_)
# print('name : ',name_)
# gain_ = gigwan.GetDataValue(1)
# fore_ = gigwan.GetDataValue(2)
# gigwan_ = gigwan.GetDataValue(3)
# print('name : ', name_)
# print('gain : ', gain_)
# print('fore : ', fore_)
# print('gigwan : ', gigwan_)
# value = stockmst.GetDataValue(2,1)

# print('value : ',value)

num :  1
thema_code :  426
code :  A000060
order :  보험


In [21]:
import win32com.client
# thema = win32com.client.Dispatch('Dscbo1.CpSvr8562')
thema = win32com.client.Dispatch('Dscbo1.CpSvr8561T')

# gigwan.SetInputValue(0,'B')
# gigwan.SetInputValue(1,0)
# gigwan.SetInputValue(2,'1')
thema.SetInputValue(0,426)

thema.BlockRequest()

code_ = thema.GetHeaderValue(0)
num_ = thema.GetHeaderValue(1)
comment_ = thema.GetHeaderValue(2)

print('code : ',code_)
print('num : ',num_)
print('comment : ',comment_)

code_ = thema.GetDataValue(0,0)
name_ = thema.GetDataValue(1,0)
now_price_ = thema.GetDataValue(2,0)
compare_ = thema.GetDataValue(3,0)
compare_rate_ = thema.GetDataValue(2,0)
purchase_amount_ = thema.GetDataValue(2,0)
compare_same_time_yesterday_price_ = thema.GetDataValue(2,0)

print('code : ',code_)
print('name : ',name_)
print('now_price : ',now_price_)
print('compare : ',compare_)
print('compare_rate : ',compare_rate_)
print('purchase_amount_ : ',purchase_amount_)
print('compare_same_time_yesterday_price_ : ',compare_same_time_yesterday_price_)

# gain_ = gigwan.GetDataValue(1)
# fore_ = gigwan.GetDataValue(2)
# gigwan_ = gigwan.GetDataValue(3)
# print('name : ', name_)
# print('gain : ', gain_)
# print('fore : ', fore_)
# print('gigwan : ', gigwan_)
# value = stockmst.GetDataValue(2,1)

# print('value : ',value)

code :  426
num :  11
comment :  
code :  A000060
name :  메리츠화재
now_price :  31650
compare :  -900
compare_rate :  31650
purchase_amount_ :  31650
compare_same_time_yesterday_price_ :  31650


In [40]:
import win32com.client
gigwan = win32com.client.Dispatch('CpSysDib.CpSvr7210T')

gigwan.SetInputValue(0,'A005930')
gigwan.SetInputValue(1,0)
# gigwan.SetInputValue(2,'1')
# gigwan.SetInputValue(4,'A005930')

gigwan.BlockRequest()

num_ = int(gigwan.GetHeaderValue(0))

print('time \tnowprice \tamount\t\tfore \t\tgigwan')

for i in range(num_) :
    time_ = gigwan.GetDataValue(0,i)
    now_price_ = gigwan.GetDataValue(1,i)
    purchase_amount_ = gigwan.GetDataValue(4,i)
    fore_ = gigwan.GetDataValue(5,i)
    gigwan_ = gigwan.GetDataValue(6,i)
    


    print('{}\t{}\t\t{}\t\t{}\t{}'.format(time_,now_price_,purchase_amount_,fore_,gigwan_))


time 	nowprice 	amount		fore 		gigwan
1430	77500		17883894		-168422	1287000
1323	78000		14395712		-175422	819000
1116	77700		11111405		-229422	592000
1000	78400		7514416		-112422	13000
930	78000		4969390		-135422	0


In [3]:
import win32com.client
import pandas as pd
secter = win32com.client.Dispatch('Dscbo1.CpSvr8561')     
item = win32com.client.Dispatch('Dscbo1.CpSvr8561T')     

df_thema = pd.DataFrame(None, columns=['thema code','thema order','thema name','item code','item name'])
secter.BlockRequest() 

num_ = secter.GetHeaderValue(0)
print('num : ',num_)
print('code \t order \t name')     

for i in range(num_) :
    

    code_ = secter.GetDataValue(0,i)
    order_ = secter.GetDataValue(1,i)
    name_ = secter.GetDataValue(2,i)

    item.SetInputValue(0,code_)
    item.BlockRequest()

    item_num_ = item.GetHeaderValue(1)

    for j in range(item_num_) :
        
        item_code_ = item.GetDataValue(0,j)
        item_name_ = item.GetDataValue(1,j)
        
        new_item = {'thema code': code_,'thema order':order_,'thema name':name_,'item code':item_code_,'item name':item_name_}
        df_thema = df_thema.append(new_item, ignore_index=True)
        print('{}\t{}\t{}\t{}\t{}'.format(code_,order_,name_,item_code_,item_name_))     

print(df_thema)



    
    




        

num :  65
code 	 order 	 name
426	1	보험	A000060	메리츠화재
426	1	보험	A000370	한화손해보험
426	1	보험	A000400	롯데손해보험
426	1	보험	A000810	삼성화재
426	1	보험	A001450	현대해상
426	1	보험	A003690	코리안리
426	1	보험	A005830	DB손해보험
426	1	보험	A032830	삼성생명
426	1	보험	A082640	동양생명
426	1	보험	A085620	미래에셋생명
426	1	보험	A088350	한화생명
425	2	증권	A001200	유진투자증권
425	2	증권	A001270	부국증권
425	2	증권	A001290	상상인증권
425	2	증권	A001720	신영증권
425	2	증권	A003460	유화증권
425	2	증권	A003470	유안타증권
425	2	증권	A003530	한화투자증권
425	2	증권	A003540	대신증권
425	2	증권	A005940	NH투자증권
425	2	증권	A006800	미래에셋증권
425	2	증권	A008560	메리츠증권
425	2	증권	A016360	삼성증권
425	2	증권	A016610	DB금융투자
425	2	증권	A030210	KTB투자증권
425	2	증권	A039490	키움증권
425	2	증권	A071050	한국금융지주
424	3	은행	A006220	제주은행
424	3	은행	A024110	기업은행
424	3	은행	A055550	신한지주
424	3	은행	A086790	하나금융지주
424	3	은행	A105560	KB금융
424	3	은행	A138930	BNK금융지주
424	3	은행	A139130	DGB금융지주
424	3	은행	A175330	JB금융지주
424	3	은행	A316140	우리금융지주
423	4	화장품	A002790	아모레G
423	4	화장품	A003350	한국화장품제조
423	4	화장품	A016100	리더스코스메틱
423	4	화장품	A018250	애경산업
423	4	화장품	A018290	브이티지엠피
423	4	화장품	A02472

In [4]:
df_thema

,thema code,thema order,thema name,item code,item name
0,426,1,보험,A000060,메리츠화재
1,426,1,보험,A000370,한화손해보험
2,426,1,보험,A000400,롯데손해보험
3,426,1,보험,A000810,삼성화재
4,426,1,보험,A001450,현대해상
...,...,...,...,...,...
672,360,65,REITs,A348950,제이알글로벌리츠
673,360,65,REITs,A350520,이지스레지던스리츠
674,360,65,REITs,A357120,코람코에너지리츠
675,360,65,REITs,A357250,미래에셋맵스리츠


In [5]:
len(df_thema.iloc[0])

5

In [10]:
for i in range(len(df_thema)) :
    for j in range(len(df_thema.iloc[i])) :
        print(i, j, df_thema.iloc[i,j])

0 0 426
0 1 1
0 2 보험
0 3 A000060
0 4 메리츠화재
1 0 426
1 1 1
1 2 보험
1 3 A000370
1 4 한화손해보험
2 0 426
2 1 1
2 2 보험
2 3 A000400
2 4 롯데손해보험
3 0 426
3 1 1
3 2 보험
3 3 A000810
3 4 삼성화재
4 0 426
4 1 1
4 2 보험
4 3 A001450
4 4 현대해상
5 0 426
5 1 1
5 2 보험
5 3 A003690
5 4 코리안리
6 0 426
6 1 1
6 2 보험
6 3 A005830
6 4 DB손해보험
7 0 426
7 1 1
7 2 보험
7 3 A032830
7 4 삼성생명
8 0 426
8 1 1
8 2 보험
8 3 A082640
8 4 동양생명
9 0 426
9 1 1
9 2 보험
9 3 A085620
9 4 미래에셋생명
10 0 426
10 1 1
10 2 보험
10 3 A088350
10 4 한화생명
11 0 425
11 1 2
11 2 증권
11 3 A001200
11 4 유진투자증권
12 0 425
12 1 2
12 2 증권
12 3 A001270
12 4 부국증권
13 0 425
13 1 2
13 2 증권
13 3 A001290
13 4 상상인증권
14 0 425
14 1 2
14 2 증권
14 3 A001720
14 4 신영증권
15 0 425
15 1 2
15 2 증권
15 3 A003460
15 4 유화증권
16 0 425
16 1 2
16 2 증권
16 3 A003470
16 4 유안타증권
17 0 425
17 1 2
17 2 증권
17 3 A003530
17 4 한화투자증권
18 0 425
18 1 2
18 2 증권
18 3 A003540
18 4 대신증권
19 0 425
19 1 2
19 2 증권
19 3 A005940
19 4 NH투자증권
20 0 425
20 1 2
20 2 증권
20 3 A006800
20 4 미래에셋증권
21 0 425
21 1 2
21 2 증권
21 3 A008560
21 4 메리

In [1]:
test_Que = [[0,0,'text0'],[0,1,'text1'],[0,2,'text2'],[0,3,'text3'],[0,4,'text4']]
for _ in test_Que :
    r,c,txt = test_Que.pop(0)
    print('r,c,txt : ',r,c,txt)

r,c,txt :  0 0 text0
r,c,txt :  0 1 text1
r,c,txt :  0 2 text2


In [4]:
import sql
sql.make_table('test4')

()

In [3]:
import win32com.client
kospi_index = win32com.client.Dispatch('Dscbo1.StockIndexIR')

kospi_index.SetInputValue(0,'U001')

kospi_index.BlockRequest()

kospi_num_ = kospi_index.GetHeaderValue(1)
print('num : ',kospi_num_)

print()

print('time \t kospi')
for i in range(kospi_num_) :
    time_ = kospi_index.GetDataValue(0,i)
    kospi_ = kospi_index.GetDataValue(1,i)

    print('{}\t{}'.format(time_, kospi_))


num :  0

time 	 kospi


In [2]:
import pymysql
sql = pymysql.connect(host = '14.35.220.97',port=3306, user='seominseok', passwd='pintech801', db='dbstock', charset='utf8')
cursor = sql.cursor()
sql_comm_ = "select 매수잔량 is not NULL from test_table where 체결시간='2021-12-14 09:00';"
cursor.execute(sql_comm_)
result_ = cursor.fetchall()
print(type(result_[0][0]))


<class 'int'>


In [2]:
import pandas as pd
import win32com.client

df_stop_code = pd.read_csv('./stop_code.csv')
stop_name = []

for i in range(len(df_stop_code)):
    stop_name.append(df_stop_code.iloc[i,1])


code_list = win32com.client.Dispatch("CpUtil.CpCodeMgr")
codeList = code_list.GetStockListByMarket(1)
kospiList = []

for code in codeList:
    secondCode = code_list.GetStockSectionKind(code)
    name = code_list.CodeToName(code)
    if ((secondCode == 1)&(name not in stop_name)):
        kospiList.append(code[1:])

df_kospi = pd.DataFrame(kospiList, columns=['code'])
df_kospi.to_csv('final_code.csv')
        
        

In [1]:
import win32com.client
gigwan = win32com.client.Dispatch('CpSysDib.StockChart')

gigwan.SetInputValue(0,'A005930')
gigwan.SetInputValue(1,ord('2'))
# gigwan.SetInputValue(2,0)
# gigwan.SetInputValue(3,20211201)
gigwan.SetInputValue(4,3)
gigwan.SetInputValue(5,[0,5,12])
gigwan.SetInputValue(6,ord('D'))

gigwan.BlockRequest()

code_ = gigwan.GetHeaderValue(0)
num_ = int(gigwan.GetHeaderValue(3))
field_name_ = gigwan.GetHeaderValue(2)
field_num_ = int(gigwan.GetHeaderValue(1))
# stock_num_ = int(gigwan.GetHeaderValue(18))

print('Code : ',code_)
print('num : ',num_)

# print('time \t gain \t fore \t gigwan')
print(field_name_)

# time_ = gigwan.GetDataValue(0,0)
# print(time_)

# for i in range(num_) :
for i in range(num_) :
    # print('i : ',i)
    for j in range(field_num_) :
        # print('j : ',j)
        time_ = gigwan.GetDataValue(j,i)
        # gain_ = gigwan.GetDataValue(1,i)
        # fore_ = gigwan.GetDataValue(2,i)
        # gigwan_ = gigwan.GetDataValue(3,i)
        print(time_, end='\t')
    print()

    # print('{}\t{}\t{}\t{}'.format(time_,gain_,fore_,gigwan_))



Code :  A005930
num :  3
('날짜', '종가', '상장주식수')
20211223	79900	5969782000	
20211222	79400	5969782000	
20211221	78100	5969782000	


In [35]:
time_ = gigwan.GetDataValue(0,0)
print(time_, end='\t')

20211223	